In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pickle
import librosa
from sklearn.preprocessing import StandardScaler

In [2]:
model_path = 'models/RandomForestClassifier_model.pkl'
with open(model_path, 'rb') as file:
    RF_model = pickle.load(file)

In [3]:
scaler_path = 'models/scaler.pkl'
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

In [4]:
class_labels = ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '8', '9']

In [5]:
class_names = ['Kanakangi', 'Natakapriya', 'Kokilapriya', 'Rupavati', '	Gayakapriya', 'Vakuḷabharaṇam', 'Mayamaḻavagowla', 'Chakravakam', '	Suryakantam', 'Hatakambari', 'Jhankaradhvani', 'Ratnangi', 'Naṭabhairavi', 'Kīravaṇi', 'Kharaharapriya', 'Gourimanohari', 'Varuṇapriya', 'Mararanjani', 'Charukesi', 'Sarasangi', 'Harikambhōji', 'Dhīrasankarabharaṇam', 'Ganamurti', 'Naganandini', 'Yagapriya', 'Ragavardhini', 'Gangeyabhusani', 'Vagadhīsvari', 'Śulini', 'Chalanaṭa', 'Salagam', 'Jalarnavam', 'Jhalavaraḷi', 'Vanaspati', 'Navanītam', 'Pavani', 'Raghupriya', 'Gavambhodi', 'Bhavapriya', 'Śubhapantuvaraḷi', 'Shaḍvidamargini', 'Suvarnangi', 'Divyamaṇi', '	Dhavaḻambari', 'Manavati', 'Namanarayaṇi', 'Kamavardhini', 'Ramapriya', 'Gamanasrama', 'Visvambari', 'Śamaḻangi', 'Śanmukhapriya', 'Simhendramadhyamam', 'Hemavati', 'Dharmavati', 'Tanarupi', 'Nītimati', 'Kantamaṇi', 'Risabhapriya', 'Latangi', 'Vachaspati', 'Mechakalyani', 'Chitrambari', 'Sucharitra', 'Jyoti svarupini', 'Dhathuvardhani', 'Senavati', 'Nasikabhusaṇi', 'Kōsalam', 'Rasikapriya', 'Hanumatodi', 'Dhenuka']

In [6]:
def extract_features(file_name):
    try:
        y, sr = librosa.load(file_name, mono=True, duration=30)

        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
        rmse = np.mean(librosa.feature.rms(y=y))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y))
        mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr), axis=1)

        features = [chroma_stft, rmse, spec_cent, spec_bw, rolloff, zcr]
        features.extend(mfcc)

        return features

    except Exception as e:
        print(f"Error processing file: {e}")

In [7]:
def predict_class(file_name):
    try:
        features = extract_features(file_name)
        if features is not None:
            features_scaled = scaler.transform([features])  
            prediction = RF_model.predict(features_scaled)

            return class_labels[prediction[0]], class_names[prediction[0]]
        else:
            return None
    except Exception as e:
        print(f"Error predicting class: {e}")
        return None

input_music_file = 'input/genres/M56/M56(4).wav'

try:
    predicted_label, predicted_names = predict_class(input_music_file)
    print("predicted_class : ", predicted_label)
    print("predicted_names : ", predicted_names)
except Exception as e:
    print(e)

predicted_class :  56
predicted_names :  Śanmukhapriya


In [8]:
if predicted_label:
    print(f"The predicted class for the input music file is: {predicted_names}")
else:
    print("Failed to predict the class of the input music file.")

The predicted class for the input music file is: Śanmukhapriya
